In [1]:
import numpy as np

# Define the RBM class
class RBM:
    def __init__(self, num_visible, num_hidden):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.learning_rate = 0.1

        # Initialize weights and biases
        self.weights = np.random.normal(0, 0.1, (num_visible, num_hidden))
        self.visible_bias = np.zeros(num_visible)
        self.hidden_bias = np.zeros(num_hidden)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def gibbs_sampling(self, visible_prob):
        hidden_prob = self.sigmoid(np.dot(visible_prob, self.weights) + self.hidden_bias)
        hidden_states = np.random.rand(self.num_hidden) < hidden_prob
        visible_prob_reconstructed = self.sigmoid(np.dot(hidden_states, self.weights.T) + self.visible_bias)
        return hidden_prob, hidden_states, visible_prob_reconstructed

    def train(self, data, num_epochs=100):
        for epoch in range(num_epochs):
            for v0 in data:
                # Positive phase
                hidden_prob0, hidden_states0, visible_prob1 = self.gibbs_sampling(v0)

                # Negative phase
                hidden_prob1, hidden_states1, visible_prob2 = self.gibbs_sampling(visible_prob1)

                self.weights += self.learning_rate * (np.outer(v0, hidden_prob0) - np.outer(visible_prob1, hidden_prob1))
                self.visible_bias += self.learning_rate * (v0 - visible_prob1)
                self.hidden_bias += self.learning_rate * (hidden_prob0 - hidden_prob1)
                #print("W after learn : " + str(self.weights) )
                #print("Vis_bias after learn : " + str(self.visible_bias) )
                #print("Hid_bias after learn : " + str(self.hidden_bias) )

    def generate_samples(self, num_samples):
        samples = np.zeros((num_samples, self.num_visible))
        for i in range(num_samples):
            visible_prob = np.random.rand(self.num_visible)
            for _ in range(1000):  # Gibbs sampling to reach equilibrium
                _, _, visible_prob = self.gibbs_sampling(visible_prob)
            samples[i] = visible_prob
        return samples

# Generate synthetic binary data
num_samples = 1000
num_visible = 2
data = np.random.randint(2, size=(num_samples, num_visible))

# Create and train the RBM
rbm = RBM(num_visible, num_hidden=2)
rbm.train(data, num_epochs=100)
print("W final learn : " + str(rbm.weights) )
print("Vis_bias final learn : " + str(rbm.visible_bias) )
print("Hid_bias final learn : " + str(rbm.hidden_bias) )
            
# Generate new samples using the trained RBM
generated_samples = rbm.generate_samples(num_samples=10)
print("Generated Samples:")
print(np.round(generated_samples))


W final learn : [[ 14.94283679  -0.40429405]
 [  0.2763981  -15.09994216]]
Vis_bias final learn : [-7.37829198  7.09225413]
Hid_bias final learn : [-7.1408988   7.39386893]
Generated Samples:
[[1. 1.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [1. 1.]]
